In [1]:
DOMAIN = 'test' # test, spouse, cdr

In [2]:
import sys
import os

if DOMAIN == 'cdr':
    os.environ['SNORKELDB'] = 'postgres:///semparse_cdr'

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 0).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 65


In [4]:
# c = candidates[0]
# print c[0].get_parent()._asdict()['entity_types']

In [5]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [6]:
# c = sv.get_selected()
# print c

In [7]:
# hash(c)

In [8]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'cdr':
    user_lists = {}

In [9]:
# examples[0].display()

In [10]:
# examples[28].candidate[0].get_parent()

In [11]:
%time from snorkel.semantic import SemanticParser

CPU times: user 1.51 s, sys: 545 ms, total: 2.06 s
Wall time: 2.42 s


In [12]:
%time sp = SemanticParser(candidate_class, user_lists, absorb=True)

Created grammar with 203 rules
CPU times: user 4.71 ms, sys: 8.86 ms, total: 13.6 ms
Wall time: 36.7 ms


In [ ]:
%time sp.evaluate(examples, show_explanation=True, show_all=False, only=[])

Example 0: label True because True
Example 1: label True because True and True
Example 2: label True because False or True
Example 3: label True because not False
Example 4: label True because 'yes' equals 'yes'
Example 5: label True because 1 is equal to 1
Example 6: label True because 1 is equal to one
Example 7: label True because 1 is less than 2
Example 8: label True because 2 is less than or equal to 2
Example 9: label True because 2 > 1
Example 10: label True because 2 is at least 2
Example 11: label True because arg 1 is lowercase
Example 12: label True because arg 1 is upper case
Example 13: label True because arg 1 is capitalized
Example 14: label True because the word 'blueberry' starts with 'blue'
Example 15: label True because the word 'blueberry' ends with 'berry'
Example 16: label True because argument 1 is 'foo'
Example 17: label True because 'bar' is in 'foobarbaz'
Example 18: label True because the word 'foobarbaz' contains 'oobarba'
Example 19: label True because 'ba

In [14]:
# (.label, .true, (.function, (.startswith, (.string, blue)), (.string, blueberry)))

In [15]:
# sp.grammar.print_grammar()

In [16]:
# sp.grammar.print_chart()

In [17]:
# total_correct = 0
# total_incorrect = 0
# total_failed = 0
# total_redundant = 0
# for i, example in enumerate(examples):
#     if i==20: break
# #     print example.explanation
#     correct = 0
#     incorrect = 0
#     failed = 0
#     redundant = 0
#     semantics = set()
#     parses = sp.parse(example.explanation, 
#                       example.candidate, 
#                       user_lists=user_lists, 
#                       verbose=False, 
#                       return_parses=True)
# 
#     total_correct += correct
#     total_incorrect += incorrect
#     total_failed += failed
#     total_redundant += redundant
#     valid = correct + incorrect
#     total = correct + incorrect + redundant + failed
#     print "Example %d: %d/%d (%d, %d) -> %d" % (i, correct, valid, redundant, failed, total)
# total_valid = total_correct + total_incorrect
# total_total = total_correct + total_incorrect + total_redundant + total_failed
# print "TOTAL: {}/{} ({:.2f}); {} redundant, {} failed".format(
#                                  total_correct, total_valid, 
#                                  float(total_correct)/total_valid, 
#                                  total_redundant, total_failed)
# print "GROWTH (total): {:.2f}".format(float(total_valid + total_faile)/len(examples))
# print "GROWTH (filter fail): {:.2f}".format(float(total_valid)/len(examples))
# print "GROWTH (filter incorrect): {:.2f}".format(float(total_correct)/len(examples))